In [16]:
import numpy as np
import pandas as pd

In [50]:
# Load dataset
input = open('InputData/a_example.in', 'r')
text = input.read()

In [51]:
# Take a look
#print(text)

In [52]:
# Load info about dataset
data_set = text.split('\n')
data_set_info = data_set[0]
data_set_info = data_set_info.split(' ')

R = int(data_set_info[0])          # number of rows of the grid 
C = int(data_set_info[1])          # number of columns of the grid 
F = int(data_set_info[2])          # number of vehicles in the fleet
N = int(data_set_info[3])          # number of rides
B = int(data_set_info[4])          # per-ride bonus for starting the ride on time
T = int(data_set_info[5])          # number of steps in the simulation

In [53]:
T

10

In [54]:
# Build numpy array to hold info about rides

def buildRides():
    rides = []
    
    for i in range(1, N + 1):
        single_ride = np.asarray(list(map(int, data_set[i].split(' '))))
        route = abs(single_ride[0] - single_ride[2]) + abs(single_ride[1] - single_ride[3])
        latest_start = single_ride[5] - route
        erliest_finish = single_ride[4] + route
        single_ride = np.append(single_ride, [route, erliest_finish, latest_start, i-1])
        rides.append(single_ride)
    rides = np.asarray(rides)
    
    return rides

In [55]:
# [a, b, x, y, s, f]

# a  – the row of the start intersection
# b  – the column of the start intersection 
# x  – the row of the finish intersection
# y  – the column of the finish intersection
# s  – the earliest start
# f  – the latest finish

# r  - route length
# ef - the earliest finish
# ls - the latest start
# id - client id

In [56]:
# Client list for taxis
def buildTaxis():
    taxi = dict()
    for i in range(F):
        taxi[i] = list()
    return taxi

In [57]:
# Taxi position
def buildTaxiPosition():
    taxi = dict()
    for i in range(F):
        taxi[i] = (0,0)
    return taxi

In [58]:
# Current rides
def buildCurrentRides():
     return dict()

In [59]:
def routeToClient(r_taxi, c_taxi, r_client, c_client):
    return abs(r_taxi - r_client) + abs(c_taxi - c_client)

In [60]:
def costFunction(nwcs, npcs, ts, dr, bonus):
#     print(nwcs)
#     print(npcs)
#     print(ts)
#     print(dr)
#     print(bonus)
    
    
    if(nwcs - ts >= dr):
        return(nwcs -ts -bonus)
    elif(npcs - ts >= dr):
        return(dr)
    else:
        return(1000000)

In [61]:
def costFunctionForRow(single_row, r_taxi, c_taxi, ts, bonus):
    r_client = single_row[0]
    c_client = single_row[1]
    dr = routeToClient(r_taxi, c_taxi, r_client, c_client)
    return costFunction(single_row[4], single_row[8], ts, dr, bonus)

In [62]:
def simulation():
    rides = buildRides()
    taxi = buildTaxis()
    taxi_position = buildTaxiPosition()
    current_rides = buildCurrentRides()
    
    for t in range(T):
        vehicle_to_remove = []
        for vehicle in current_rides:
            if(current_rides[vehicle][-1] == t):
                taxi_position[vehicle] = (current_rides[vehicle][1], current_rides[vehicle][2])
                taxi[vehicle].append(current_rides[vehicle][0])
                vehicle_to_remove.append(vehicle)
        
        for vehicle in vehicle_to_remove:
            del current_rides[vehicle]
        
        for vehicle in range(F):
            if(vehicle not in current_rides.keys()):
                if(rides.shape[0] != 0):
#                     cost_values = np.array([])
#                     for row in rides:
#                         cost_values = np.append(cost_values, costFunctionForRow(row, taxi_position[vehicle][0], taxi_position[vehicle][1], t, B))
                    cost_values = np.apply_along_axis(costFunctionForRow, 1, rides, taxi_position[vehicle][0], taxi_position[vehicle][1], t, B)
                    tmp_index = np.argmin(cost_values)
                    client = rides[tmp_index]
                    client_id = client[-1]
                    client_row_finish = client[2]
                    client_column_finish = client[3]
                    time_finish = client[6] + t

                    current_rides[vehicle] = (client_id, client_row_finish, client_column_finish, time_finish)
                    rides = np.delete(rides, tmp_index, 0)
    return taxi 

In [63]:
sim_output = simulation()

In [64]:
print(sim_output)

{0: [0], 1: [2, 1]}


In [65]:
output = ""
for line in sim_output:
    single = str(len(sim_output[line]))
    for val in sim_output[line]:
        single += " "
        single += str(val)
    single += '\n'
    output += single
output = output[:-1]
print(output)

1 0
2 2 1


In [66]:
text_file = open("OutputData/a_example.out", "w+")

text_file.write(output)

text_file.close()